In [318]:
import nba_py
from nba_py import player
from nba_py import constants
import requests
import json
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import re

In [329]:
basepath='https://www.basketball-reference.com'
endpath='/playoffs/1981-nba-finals-rockets-vs-celtics.html'
#https://www.basketball-reference.com/playoffs/
fullpath=basepath+endpath
mvps=[]
champs=[]
years=[]
hrefss=[]
year=1981

while endpath!=None:
    soup=BeautifulSoup(requests.get(fullpath).text,'html.parser')

    ps=soup.find_all('p')
    for p in ps:
        #print(p)
        if (p.text.find('Finals MVP')!=-1):
            mvp=p.find('a').text
        if (p.text.find('League Champion')!=-1):
            champ=p.find('a').text

    

    tabs=soup.find('div',{'id':'content'}).findAll('table')
    hrefs=[]
    for t in tabs:
        aas=t.findAll('a')
        #print(aas)
        for aa in aas:
            if aa.text=='Final':
                hrefs.append(aa['href'])
    hrefss.append(hrefs)
    
    
    mvps.append(mvp)
    champs.append(champ)
    years.append(year)
    year+=1
    endpath=soup.findAll('a',{'class':'next'})
    #if len(endpath)>0:
    endpath=endpath[0]['href']
    fullpath=basepath+endpath

IndexError: list index out of range

In [327]:
soup.findAll('a')

[<a href="https://www.sports-reference.com/"><svg height="15px" width="20px"><use xlink:href="#ic-sr-pennant"></use></svg> Sports Reference</a>,
 <a href="https://www.baseball-reference.com/">Baseball</a>,
 <a href="https://www.pro-football-reference.com/">Football</a>,
 <a href="https://www.sports-reference.com/cfb/">(college)</a>,
 <a href="https://www.basketball-reference.com/">Basketball</a>,
 <a href="https://www.sports-reference.com/cbb/">(college)</a>,
 <a href="https://www.hockey-reference.com/">Hockey</a>,
 <a href="https://fbref.com/">Soccer</a>,
 <a href="https://www.sports-reference.com/blog/">Blog</a>,
 <a href="https://stathead.com/">Stathead</a>,
 <a href="https://widgets.sports-reference.com/">Widgets</a>,
 <a href="https://www.sports-reference.com/feedback/">Questions or Comments?</a>,
 <a href="/my/">Your Account</a>,
 <a class="logout" href="/my/?do=logout">Logout</a>,
 <a class="login" href="/my/">Login</a>,
 <a href="/my/">Create Account</a>,
 <a href="/my/?do=ad_f

In [330]:
print(mvps)
print(years)
print(champs)
#print(hrefss)

['Cedric Maxwell', 'Magic Johnson', 'Moses Malone', 'Larry Bird', 'Kareem Abdul-Jabbar', 'Larry Bird', 'Magic Johnson', 'James Worthy', 'Joe Dumars', 'Isiah Thomas', 'Michael Jordan', 'Michael Jordan', 'Michael Jordan', 'Hakeem Olajuwon', 'Hakeem Olajuwon', 'Michael Jordan', 'Michael Jordan', 'Michael Jordan', 'Tim Duncan', "Shaquille O'Neal", "Shaquille O'Neal", "Shaquille O'Neal", 'Tim Duncan', 'Chauncey Billups', 'Tim Duncan', 'Dwyane Wade', 'Tony Parker', 'Paul Pierce', 'Kobe Bryant', 'Kobe Bryant', 'Dirk Nowitzki', 'LeBron James', 'LeBron James', 'Kawhi Leonard', 'Andre Iguodala', 'LeBron James', 'Kevin Durant', 'Kevin Durant']
[1981, 1982, 1983, 1984, 1985, 1986, 1987, 1988, 1989, 1990, 1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018]
['Boston Celtics', 'Los Angeles Lakers', 'Philadelphia 76ers', 'Boston Celtics', 'Los Angeles Lakers', 'Boston Celtics', 'Los Ange

In [331]:
#endpath='/boxscores/197805210SEA.html'
#print(soup.prettify())
basepath='https://www.basketball-reference.com'
tabledats=[]
allheaders=[]
allplayers=[]
count=0
for mvp,year,champ,hrefs in zip(mvps,years,champs,hrefss):  
    tabledat=[]
    game=0
    for href in hrefs:
        game+=1
        fullpath=basepath+href
        soup=BeautifulSoup(requests.get(fullpath).text,'html.parser')

        tabs=soup.findAll('table')
        #rowdat=[]

        for tab in tabs:
            #print(tab)
            if tab.has_attr('id'):
                if tab['id'].find('basic')!=-1:
                    team=tab.find('caption').text.split('(')[0].strip()
                    if count==0:
                        headers=tab.find_all('th',{'class':' poptip center'})
                        headers=[header.text.strip() for header in headers]
                        allheaders.append(headers)
                    players=tab.find_all('th',{'class':'left '})
                    players=[[player.text.strip(),team] for player in players]
                    
                    allplayers+=players
                    rows=tab.find_all('tr')
                    for row in rows:
                        cols=row.find_all('td')
                        cols=[ele.text.strip() for ele in cols]
                        rowdat=[ele for ele in cols]
                        if len(rowdat)>0:
                            if len(rowdat)>19:
                                rowdat=rowdat[:19]
                            rowdat.append(game)
                            rowdat.append(year)
            #i.append(winteam)
                            rowdat.append(champ)
                            rowdat.append(mvp)
                            tabledat.append(rowdat)
                            #tabledat.append(game)
                    

        #allplayers[0]=[list(i).append(winteam) for i in allplayers[0]]
        #allplayers[1]=[list(i).append(loseteam) for i in allplayers[1]]
        #print(count)
        if count==0:
            header=allheaders[0]
        #print(header)
            colcount=len(header)
        #print(cols)
        #print(colcount)
        
#         for i in range(len(tabledat)):
#             row=tabledat[i]
#             #print(row)
#             if len(row)>colcount:
#                 #print(len(i))
#                 row=row[:colcount]
#             #row.append(game)
#             row.append(year)
#             #i.append(winteam)
#             row.append(champ)
#             row.append(mvp)
#             #print(i)
#             tabledat[i]=row
        
        

        #print(tabledat)
        if count==0: 
            header.insert(0,'Team')
            header.insert(0,'Player')
            header.append('Game')
            header.append('Year')
        #header.append('WinGTeam')
            header.append('Champ')
            header.append('MVP')
            count+=1
    tabledats+=tabledat
for i,j in zip(allplayers,tabledats):
    i+=j

In [332]:
#print(allplayers)
print(header)
#print(tabledat)

#print(allplayers)
allfin=pd.DataFrame(allplayers,columns=header)


print(allfin.shape)
allfin.head(50)

['Player', 'Team', 'MP', 'FG', 'FGA', 'FG%', '3P', '3PA', '3P%', 'FT', 'FTA', 'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'Game', 'Year', 'Champ', 'MVP']
(4816, 25)


,Player,Team,MP,FG,FGA,FG%,3P,3PA,3P%,FT,...,AST,STL,BLK,TOV,PF,PTS,Game,Year,Champ,MVP
0,Robert Reid,Houston Rockets,48:00,12,22,.545,0,1,.000,3,...,3,4,0,2,2,27,1.0,1981.0,Boston Celtics,Cedric Maxwell
1,Moses Malone,Houston Rockets,45:00,4,17,.235,0,0,,5,...,3,3,1,1,1,13,1.0,1981.0,Boston Celtics,Cedric Maxwell
2,Billy Paultz,Houston Rockets,37:00,7,16,.438,0,0,,0,...,4,3,1,3,2,14,1.0,1981.0,Boston Celtics,Cedric Maxwell
3,Mike Dunleavy,Houston Rockets,37:00,2,9,.222,0,0,,2,...,7,0,0,1,3,6,1.0,1981.0,Boston Celtics,Cedric Maxwell
4,Tom Henderson,Houston Rockets,22:00,5,8,.625,0,0,,1,...,3,0,0,0,4,11,1.0,1981.0,Boston Celtics,Cedric Maxwell
5,Calvin Murphy,Houston Rockets,24:00,8,15,.533,0,0,,0,...,0,2,0,1,5,16,1.0,1981.0,Boston Celtics,Cedric Maxwell
6,Allen Leavell,Houston Rockets,13:00,3,7,.429,0,0,,0,...,3,1,0,1,1,6,1.0,1981.0,Boston Celtics,Cedric Maxwell
7,Bill Willoughby,Houston Rockets,13:00,1,4,.250,0,0,,0,...,0,2,1,1,2,2,1.0,1981.0,Boston Celtics,Cedric Maxwell
8,Rudy Tomjanovich,Houston Rockets,1:00,0,1,.000,0,1,.000,0,...,0,0,0,0,0,0,1.0,1981.0,Boston Celtics,Cedric Maxwell
9,Team Totals,Houston Rockets,240,42,99,.424,0,2,.000,11,...,23,15,3,10,20,95,1.0,1981.0,Boston Celtics,Cedric Maxwell


In [333]:
allfin.tail(50)

,Player,Team,MP,FG,FGA,FG%,3P,3PA,3P%,FT,...,AST,STL,BLK,TOV,PF,PTS,Game,Year,Champ,MVP
4766,Andre Iguodala,Golden State Warriors,21:53,3,4,.750,0,0,,2,...,1,1,0,1,3,8,3.0,2018.0,Golden State Warriors,Kevin Durant
4767,Shaun Livingston,Golden State Warriors,17:23,4,5,.800,0,0,,0,...,2,0,0,0,3,8,3.0,2018.0,Golden State Warriors,Kevin Durant
4768,Jordan Bell,Golden State Warriors,11:58,4,5,.800,0,0,,2,...,0,0,1,0,0,10,3.0,2018.0,Golden State Warriors,Kevin Durant
4769,David West,Golden State Warriors,5:04,0,2,.000,0,0,,0,...,0,0,0,1,0,0,3.0,2018.0,Golden State Warriors,Kevin Durant
4770,Nick Young,Golden State Warriors,4:48,0,0,,0,0,,0,...,0,0,0,0,0,0,3.0,2018.0,Golden State Warriors,Kevin Durant
4771,Patrick McCaw,Golden State Warriors,0:58,0,0,,0,0,,0,...,0,0,0,0,0,0,3.0,2018.0,Golden State Warriors,Kevin Durant
4772,Kevon Looney,Golden State Warriors,0:01,0,0,,0,0,,0,...,0,0,0,0,0,0,3.0,2018.0,Golden State Warriors,Kevin Durant
4773,Zaza Pachulia,Golden State Warriors,Did Not Play,3,2018,Golden State Warriors,Kevin Durant,None,None,None,...,None,None,None,None,None,None,NaN,NaN,None,None
4774,Team Totals,Golden State Warriors,240,42,81,.519,9,26,.346,17,...,27,6,5,10,20,110,3.0,2018.0,Golden State Warriors,Kevin Durant
4775,LeBron James,Cleveland Cavaliers,46:52,13,28,.464,1,6,.167,6,...,11,2,2,4,2,33,3.0,2018.0,Golden State Warriors,Kevin Durant


In [334]:
allfin=allfin[allfin.Year.notnull()]
allfin[allfin.Year==2018]

,Player,Team,MP,FG,FGA,FG%,3P,3PA,3P%,FT,...,AST,STL,BLK,TOV,PF,PTS,Game,Year,Champ,MVP
4705,LeBron James,Cleveland Cavaliers,47:32,19,32,.594,3,7,.429,10,...,8,1,1,5,2,51,1.0,2018.0,Golden State Warriors,Kevin Durant
4706,Kevin Love,Cleveland Cavaliers,39:04,9,20,.450,1,8,.125,2,...,1,0,0,1,1,21,1.0,2018.0,Golden State Warriors,Kevin Durant
4707,J.R. Smith,Cleveland Cavaliers,38:59,3,10,.300,2,6,.333,2,...,2,0,0,2,1,10,1.0,2018.0,Golden State Warriors,Kevin Durant
4708,George Hill,Cleveland Cavaliers,30:29,2,6,.333,2,4,.500,1,...,1,1,0,0,5,7,1.0,2018.0,Golden State Warriors,Kevin Durant
4709,Tristan Thompson,Cleveland Cavaliers,20:16,1,4,.250,0,0,,0,...,0,0,1,1,3,2,1.0,2018.0,Golden State Warriors,Kevin Durant
4710,Jeff Green,Cleveland Cavaliers,35:58,3,9,.333,1,6,.167,0,...,5,1,0,1,1,7,1.0,2018.0,Golden State Warriors,Kevin Durant
4711,Larry Nance,Cleveland Cavaliers,19:21,4,6,.667,0,0,,1,...,0,1,0,0,2,9,1.0,2018.0,Golden State Warriors,Kevin Durant
4712,Jordan Clarkson,Cleveland Cavaliers,17:09,2,9,.222,0,3,.000,0,...,0,1,1,1,1,4,1.0,2018.0,Golden State Warriors,Kevin Durant
4713,Kyle Korver,Cleveland Cavaliers,16:01,1,3,.333,1,3,.333,0,...,1,0,0,0,2,3,1.0,2018.0,Golden State Warriors,Kevin Durant
4714,Cedi Osman,Cleveland Cavaliers,0:03,0,0,,0,0,,0,...,0,0,0,0,0,0,1.0,2018.0,Golden State Warriors,Kevin Durant


In [335]:
allfin.iloc[:,2]=allfin.iloc[:,2].apply(lambda x:x.split(':')[0])
allfin.iloc[:,2:20]=allfin.iloc[:,2:20].apply(lambda x:pd.to_numeric(x,errors='coerce'))
allfin=allfin.fillna(value=0)

In [336]:
allfin['GS']=(allfin['PTS'].astype(float)+0.4*allfin['FG']-0.7*allfin['FGA']-0.4*(allfin['FTA']-allfin['FT'])+
0.7*allfin['ORB']+0.3*allfin['DRB']+allfin['STL']+0.7*allfin['AST']+0.7*allfin['BLK']-
0.4*allfin['PF']-allfin['TOV'])


In [346]:
allfin['GSGRank']=allfin.groupby(['Year','Game'])['GS'].rank(ascending=False)
allfin['GSARank']=allfin['GS'].rank(ascending=False)

In [337]:
allfin=allfin[allfin['Player']!='Team Totals']

In [347]:
#yeargroup=allfin.groupby(['Year','Player','MVP','Team','Champ'],as_index=False).aggregate([min,max,np.mean],as_index=False)
yeargroup=allfin.groupby(['Year','Player','MVP','Team','Champ'],as_index=False).mean()
yeargroup['avGSRank']=yeargroup.groupby(['Year','MVP','Champ'])['GS'].rank(ascending=False)
yeargroup

,Year,Player,MVP,Team,Champ,MP,FG,FGA,FG%,3P,...,AST,STL,BLK,TOV,PF,Game,GS,GSGRank,GSARank,avGSRank
0,1981.0,Allen Leavell,Cedric Maxwell,Houston Rockets,Boston Celtics,14.600000,2.000000,6.600000,0.270200,0.000000,...,3.400000,0.600000,0.600000,1.000000,1.800000,3.400000,3.360000,13.200000,2749.600000,15.0
1,1981.0,Bill Willoughby,Cedric Maxwell,Houston Rockets,Boston Celtics,22.333333,2.333333,8.666667,0.258833,0.000000,...,1.166667,1.166667,1.000000,1.333333,2.666667,3.500000,4.466667,11.916667,2522.916667,13.0
2,1981.0,Billy Paultz,Cedric Maxwell,Houston Rockets,Boston Celtics,32.666667,5.166667,12.833333,0.397667,0.000000,...,1.333333,1.166667,0.666667,1.500000,2.666667,3.500000,6.716667,9.166667,2010.666667,7.0
3,1981.0,Calvin Garrett,Cedric Maxwell,Houston Rockets,Boston Celtics,10.750000,1.250000,2.250000,0.437500,0.000000,...,0.250000,0.500000,0.000000,0.250000,0.750000,4.000000,2.675000,13.000000,2869.625000,17.0
4,1981.0,Calvin Murphy,Cedric Maxwell,Houston Rockets,Boston Celtics,22.000000,4.250000,10.750000,0.385250,0.000000,...,0.750000,1.000000,0.000000,2.000000,2.500000,2.750000,2.750000,14.250000,2825.125000,16.0
5,1981.0,Cedric Maxwell,Cedric Maxwell,Boston Celtics,Boston Celtics,37.833333,7.000000,12.333333,0.553167,0.000000,...,2.833333,0.166667,1.000000,3.166667,3.500000,3.500000,14.766667,6.166667,1211.916667,3.0
6,1981.0,Chris Ford,Cedric Maxwell,Boston Celtics,Boston Celtics,30.666667,4.500000,9.000000,0.489833,0.333333,...,2.500000,0.666667,0.000000,1.333333,1.833333,3.500000,7.483333,8.333333,1806.000000,6.0
7,1981.0,Eric Fernsten,Cedric Maxwell,Boston Celtics,Boston Celtics,2.500000,0.000000,0.750000,0.000000,0.000000,...,0.250000,0.250000,0.000000,0.000000,0.500000,4.500000,0.500000,15.750000,3416.500000,21.0
8,1981.0,Gerald Henderson,Cedric Maxwell,Boston Celtics,Boston Celtics,15.833333,3.166667,6.000000,0.456333,0.000000,...,1.666667,0.666667,0.333333,0.666667,1.666667,3.500000,5.733333,11.000000,2254.666667,8.0
9,1981.0,Kevin McHale,Cedric Maxwell,Boston Celtics,Boston Celtics,13.833333,2.000000,5.166667,0.423833,0.000000,...,0.833333,0.000000,0.666667,1.000000,2.166667,3.500000,2.500000,13.000000,2884.250000,18.0


In [351]:
invyears=yeargroup[(yeargroup['avGSRank']==1) & (yeargroup['Team']!=yeargroup['Champ'])].sort_values('Year')[['Year','Player','MVP']]
yeargroup[(yeargroup['avGSRank']==1) & (yeargroup['Team']!=yeargroup['Champ'])].sort_values('Year')


,Year,Player,MVP,Team,Champ,MP,FG,FGA,FG%,3P,...,AST,STL,BLK,TOV,PF,Game,GS,GSGRank,GSARank,avGSRank
14,1981.0,Moses Malone,Cedric Maxwell,Houston Rockets,Boston Celtics,44.833333,8.000000,19.833333,0.414333,0.000000,...,1.333333,0.833333,2.166667,2.000000,1.833333,3.5,18.833333,2.166667,544.250000,1.0
34,1982.0,Julius Erving,Magic Johnson,Philadelphia 76ers,Los Angeles Lakers,38.333333,9.500000,17.500000,0.552167,0.000000,...,3.333333,1.833333,1.333333,3.166667,2.000000,3.5,20.466667,2.500000,435.333333,1.0
215,1990.0,Clyde Drexler,Isiah Thomas,Portland Trail Blazers,Detroit Pistons,40.800000,10.200000,18.800000,0.542800,0.400000,...,6.200000,1.800000,0.200000,3.400000,4.000000,3.0,21.260000,2.400000,466.900000,1.0
359,1996.0,Shawn Kemp,Michael Jordan,Seattle SuperSonics,Chicago Bulls,40.333333,8.166667,14.833333,0.555833,0.000000,...,2.166667,1.333333,2.000000,4.000000,4.666667,3.5,18.866667,2.833333,610.333333,1.0
546,2004.0,Shaquille O'Neal,Chauncey Billups,Los Angeles Lakers,Detroit Pistons,42.600000,10.600000,16.800000,0.622600,0.000000,...,1.600000,0.400000,0.600000,2.800000,4.400000,3.0,18.660000,3.000000,707.700000,1.0
555,2005.0,Chauncey Billups,Tim Duncan,Detroit Pistons,San Antonio Spurs,40.142857,6.571429,15.142857,0.431714,1.571429,...,6.285714,1.000000,0.142857,1.285714,2.857143,4.0,17.314286,3.428571,754.785714,1.0
696,2011.0,Dwyane Wade,Dirk Nowitzki,Miami Heat,Dallas Mavericks,38.666667,9.833333,18.000000,0.536667,1.166667,...,5.166667,1.500000,1.500000,2.500000,2.333333,3.5,22.666667,2.500000,393.333333,1.0
775,2014.0,LeBron James,Kawhi Leonard,Miami Heat,San Antonio Spurs,37.200000,10.400000,18.200000,0.574400,2.800000,...,4.000000,2.000000,0.400000,3.800000,2.000000,3.0,22.520000,1.600000,307.800000,1.0
806,2015.0,LeBron James,Andre Iguodala,Cleveland Cavaliers,Golden State Warriors,45.166667,13.000000,32.666667,0.392167,2.166667,...,8.833333,1.333333,0.500000,3.500000,3.166667,3.5,24.600000,2.166667,310.000000,1.0
887,2018.0,LeBron James,Kevin Durant,Cleveland Cavaliers,Golden State Warriors,44.000000,12.250000,23.250000,0.524000,1.500000,...,10.000000,1.250000,1.000000,5.000000,2.750000,2.5,28.325000,1.750000,206.750000,1.0


In [363]:
invdats=pd.DataFrame()
for i,dat in invyears.iterrows():
    players=list(dat[['Player','MVP']])
    print(players)
    invdat=allfin[(allfin['Year']==dat['Year'])&(allfin['Player'].isin(players))]
    invdats=invdats.append(invdat)

['Moses Malone', 'Cedric Maxwell']
['Julius Erving', 'Magic Johnson']
['Clyde Drexler', 'Isiah Thomas']
['Shawn Kemp', 'Michael Jordan']
["Shaquille O'Neal", 'Chauncey Billups']
['Chauncey Billups', 'Tim Duncan']
['Dwyane Wade', 'Dirk Nowitzki']
['LeBron James', 'Kawhi Leonard']
['LeBron James', 'Andre Iguodala']
['LeBron James', 'Kevin Durant']


In [366]:
invdats.sort_values(['Year','Game','Player'])


,Player,Team,MP,FG,FGA,FG%,3P,3PA,3P%,FT,...,TOV,PF,PTS,Game,Year,Champ,MVP,GS,GSGRank,GSARank
13,Cedric Maxwell,Boston Celtics,34,4,12,0.333,0,0,0.000,2,...,4,4,10,1.0,1981.0,Boston Celtics,Cedric Maxwell,6.5,10.0,1850.0
1,Moses Malone,Houston Rockets,45,4,17,0.235,0,0,0.000,5,...,1,1,13,1.0,1981.0,Boston Celtics,Cedric Maxwell,14.4,3.0,836.0
31,Cedric Maxwell,Boston Celtics,37,3,8,0.375,0,0,0.000,0,...,3,4,6,2.0,1981.0,Boston Celtics,Cedric Maxwell,0.4,17.0,3384.0
20,Moses Malone,Houston Rockets,48,10,24,0.417,0,0,0.000,11,...,2,1,31,2.0,1981.0,Boston Celtics,Cedric Maxwell,23.4,1.0,230.0
41,Cedric Maxwell,Boston Celtics,36,9,16,0.563,0,0,0.000,1,...,4,4,19,3.0,1981.0,Boston Celtics,Cedric Maxwell,13.4,4.0,931.5
52,Moses Malone,Houston Rockets,40,7,13,0.538,0,0,0.000,9,...,1,1,23,3.0,1981.0,Boston Celtics,Cedric Maxwell,25.2,1.0,163.0
65,Cedric Maxwell,Boston Celtics,41,9,14,0.643,0,0,0.000,6,...,4,2,24,4.0,1981.0,Boston Celtics,Cedric Maxwell,21.0,1.0,331.0
76,Moses Malone,Houston Rockets,48,11,30,0.367,0,0,0.000,2,...,3,4,24,4.0,1981.0,Boston Celtics,Cedric Maxwell,13.0,4.0,970.5
95,Cedric Maxwell,Boston Celtics,39,10,13,0.769,0,0,0.000,8,...,1,3,28,5.0,1981.0,Boston Celtics,Cedric Maxwell,31.7,1.0,44.0
83,Moses Malone,Houston Rockets,41,7,14,0.500,0,0,0.000,6,...,4,2,20,5.0,1981.0,Boston Celtics,Cedric Maxwell,15.1,2.0,769.0


In [368]:
invdats.to_csv('invdats.csv')

In [28]:
#print(soup.findAll('h2'))

print(winteam)
print(loseteam)

Cleveland Cavaliers
Golden State Warriors


In [26]:
soup.findAll('a',{'class':'next'})

[]

In [14]:
print(re.search(r'\([\d]-[\d]\)','yes   (1-0)'))

<_sre.SRE_Match object; span=(6, 11), match='(1-0)'>


In [ ]:
win=0
h2s=soup.findAll('h2')
#print(h2s)
for h2 in h2s:
    #print(h2)
    h2=h2.text
    #print(h2.text)
    if len(h2)>0:
        if re.search('\([\d]-[\d]\)',h2) is not None:
            #print(h2)
            #print(i)
            if win==0:
                winteam=h2.rsplit(' ',1)[0]
                win+=1
            else:
                loseteam=h2.rsplit(' ',1)[0]